In [27]:
import pandas as pd
import numpy as np
import requests
import tweepy
import json

import time

# data gathering
* read from `twitter-archive-enhanced.csv`
* download `image_predictions.tsv`

* use tweepy to load
    - retweet count
    - like count
    - ...
* save tweater data to `tweet_json.txt` and load them as df

#### read from `twitter-archive-enhanced.csv`

In [28]:
twitter_acrchive = pd.read_csv('twitter-archive-enhanced.csv')

#### download `image_predictions.tsv`

In [29]:
response = requests.get('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv')

In [30]:
with open('image_predictions.tsv','wb') as file:
    file.write(response.content)

image_predictions = pd.read_csv('image_predictions.tsv',sep='\t')

* use tweepy to load
    - retweet count
    - like count
    - ...


In [ ]:
# consumer_key = 'YOUR CONSUMER KEY'
# consumer_secret = 'YOUR CONSUMER SECRET'
# access_token = 'YOUR ACCESS TOKEN'
# access_secret = 'YOUR ACCESS SECRET'

# auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(access_token, access_secret)

## http://docs.tweepy.org/en/v3.2.0/api.html#API

# api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

- https://tweepy.readthedocs.io/en/latest/api.html#API.get_status
- http://docs.tweepy.org/en/latest/extended_tweets.html?highlight=tweet_mode#extended-mode
- http://docs.tweepy.org/en/latest/extended_tweets.html?highlight=tweet_mode#examples

In [ ]:
# for tweet_id in tweater_acrchive.tweet_id:
#     tweet = api.get_status(tweet_id, tweet_mode='extended')
#     print(tweet.text)
#     break

* save tweater data to `tweet-json.txt` 
* convert data to dict then to df

In [ ]:
## https://stackabuse.com/reading-and-writing-json-to-a-file-in-python/
# with open(tweet-json.txt,'wb') as file:
#     json.dump()

In [31]:
## temporary
rows = []
with open('tweet-json.txt','r') as json_file:
    for line in json_file.readlines():
        data = json.loads(line)
        data['favorite_count']
        
        rows.append({'tweet_id': data['id'],
                     'retweet_count':  data['retweet_count'],
                     'favorite_count': data['favorite_count'],
                     'hashtags': data['entities']['hashtags']})

In [32]:
twitter_extra = pd.DataFrame(rows)

### save loaded data to csv

In [52]:
image_predictions.to_csv('image_predictions.csv')
twitter_extra.to_csv('twitter_extra.csv')

# Assessing Data

### check any error in rating , dog stages

quate
```
I extracted this data programmatically, but I didn't do a very good job. The ratings probably aren't all correct. Same goes for the dog names and probably dog stages .
```




In [34]:
calculated_score =  twitter_acrchive.rating_numerator.astype(str) + '/' + tweater_acrchive.rating_denominator.astype(str)

In [35]:
tweete_score = tweater_acrchive.text.str.extract(r'(\d+\/\d+)')[0]

In [36]:
twitter_acrchive[calculated_score!=tweate_score].values

array([[835246439529840640, 8.35245984028504e+17, 26259576.0,
        '2017-02-24 21:54:03 +0000',
        '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
        "@jonnysun @Lin_Manuel ok jomny I know you're excited but 960/00 isn't a valid rating, 13/10 is tho",
        nan, nan, nan, nan, 960, 0, 'None', 'None', 'None', 'None',
        'None'],
       [826598799820865537, 8.265983652700078e+17, 4196983835.0,
        '2017-02-01 01:11:25 +0000',
        '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
        "I was going to do 007/10, but the joke wasn't worth the &lt;10 rating",
        nan, nan, nan, nan, 7, 10, 'None', 'None', 'None', 'None',
        'None']], dtype=object)

* `835246439529840640` ratings at end of text

* `826598799820865537` is was comment , was rated in other post

https://twitter.com/dog_rates/status/826598799820865537

In [37]:
twitter_acrchive[['rating_numerator','rating_denominator']].describe()

,rating_numerator,rating_denominator
count,2356.000000,2356.000000
mean,13.126486,10.455433
std,45.876648,6.745237
min,0.000000,0.000000
25%,10.000000,10.000000
50%,11.000000,10.000000
75%,12.000000,10.000000
max,1776.000000,170.000000


In [38]:
small_rate = tweater_acrchive[tweater_acrchive.rating_numerator < 10]

In [39]:
small_rate[['text','rating_numerator','rating_denominator']].values

array([['This is Bella. She hopes her smile made you smile. If not, she is also offering you her favorite monkey. 13.5/10 https://t.co/qjrljjt948',
        5, 10],
       ["This is Jerry. He's doing a distinguished tongue slip. Slightly patronizing tbh. You think you're better than us, Jerry? 6/10 hold me back https://t.co/DkOBbwulw1",
        6, 10],
       ["When you're so blinded by your systematic plagiarism that you forget what day it is. 0/10 https://t.co/YbEJPkg4Ag",
        0, 10],
       ...,
       ['Here is a very happy pup. Big fan of well-maintained decks. Just look at that tongue. 9/10 would cuddle af https://t.co/y671yMhoiR',
        9, 10],
       ['This is a western brown Mitsubishi terrier. Upset about leaf. Actually 2 dogs here. 7/10 would walk the shit out of https://t.co/r7mOb2m0UI',
        7, 10],
       ['Here we have a Japanese Irish Setter. Lost eye in Vietnam (?). Big fan of relaxing on stair. 8/10 would pet https://t.co/BLDqew2Ijj',
        8, 10]], dtype=ob

* rating with fraction has mistakes

In [40]:
small_dem = twitter_acrchive[tweater_acrchive.rating_denominator < 10]
small_dem[['tweet_id','text','rating_numerator','rating_denominator']].values

array([[835246439529840640,
        "@jonnysun @Lin_Manuel ok jomny I know you're excited but 960/00 isn't a valid rating, 13/10 is tho",
        960, 0],
       [810984652412424192,
        'Meet Sam. She smiles 24/7 &amp; secretly aspires to be a reindeer. \nKeep Sam smiling by clicking and sharing this link:\nhttps://t.co/98tB8y7y7t https://t.co/LouL5vdvxx',
        24, 7],
       [666287406224695296,
        'This is an Albanian 3 1/2 legged  Episcopalian. Loves well-polished hardwood flooring. Penis on the collar. 9/10 https://t.co/d9NcXFKwLv',
        1, 2]], dtype=object)

* 810984652412424192 not rating 
* 666287406224695296 rating at the end

In [41]:
##  already reported
big_dem = tweater_acrchive[tweater_acrchive.rating_numerator > 20]
big_dem[['tweet_id','text','rating_numerator','rating_denominator']].values

array([[855862651834028034,
        '@dhmontgomery We also gave snoop dogg a 420/10 but I think that predated your research',
        420, 10],
       [855860136149123072,
        '@s8n You tried very hard to portray this good boy as not so good, but you have ultimately failed. His goodness shines through. 666/10',
        666, 10],
       [838150277551247360, '@markhoppus 182/10', 182, 10],
       [835246439529840640,
        "@jonnysun @Lin_Manuel ok jomny I know you're excited but 960/00 isn't a valid rating, 13/10 is tho",
        960, 0],
       [832215909146226688,
        "RT @dog_rates: This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wu…",
        75, 10],
       [820690176645140481,
        'The floofs have been released I repeat the floofs have been released. 84/70 https://t.co/NIYC820tmd',
        84, 70],
       [810984652412424192,
        'Meet Sam. She smiles 24/7 &amp; secretly aspires to be a rei

### check other column

In [42]:
twitter_acrchive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


- complex values[url, string] in source coloumn 

In [43]:
tweater_acrchive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

- stages`[doggo , floofer ..]` and name columns use string `None` or `none` to represent null value
- wrong dtypes: `[timestamp,retweeted_status_timestamp]`
- dog type in represented in multiple columns (doggo  floofer pupper puppo)
- column source is not descriptive


In [ ]:
image_predictions.head()

In [ ]:
image_predictions.info()

# ROUND 1
### messy
- tweet extra has related data to tweater_archive
- dog type in represented in column name name (doggo  floofer pupper puppo)
- column source is not descriptive


### dirty
* tweater_archive
    - complex values in source coloumn 
    - tweat `[826598799820865537 , 810984652412424192 ]` is not rating tweete
    - rating with fraction takes fraction only _ex.. `9.5/10` is saved as `5/10`_
    - tweets with rating like value at begining and right rating at the end 
    ex..` [835246439529840640, 666287406224695296] `
    - `181` tweats that have retweet_status
    ```
    You only want original ratings (no retweets) that have images. Though there are 5000+ tweets in the dataset, not all are dog ratings and some are retweets.
    ```
    - stages[doggo , floofer ..] and name columns use string `None` or `none` to represent null value

    - wrong dtypes
        * tweater_archive: timestamp,retweeted_status_timestamp

# ROUND 2 _after cleaning
### messy
- some dogs has multipe types


    
# TODO assess images

# cleanning

## make copy

In [146]:
twitter_acrchive_clean =  twitter_acrchive.copy()
twitter_extra_clean = twitter_extra.copy()
image_predictions_clean = image_predictions.copy()

## tidness issues

### tweet extra has related data to twitter_archive

solution:
* merge the two data frames with tweet_id

In [147]:
twitter_acrchive_clean = twitter_acrchive_clean.merge(twitter_extra, on='tweet_id')


## test

In [148]:
twitter_acrchive_clean.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2354 entries, 0 to 2353
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2354 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2354 non-null   object 
 4   source                      2354 non-null   object 
 5   text                        2354 non-null   object 
 6   retweeted_status_id         179 non-null    float64
 7   retweeted_status_user_id    179 non-null    float64
 8   retweeted_status_timestamp  179 non-null    object 
 9   expanded_urls               2295 non-null   object 
 10  rating_numerator            2354 non-null   int64  
 11  rating_denominator          2354 non-null   int64  
 12  name                        2354 non-null   object 
 13  doggo                       2354 

### dog type is represented in column name (doggo  floofer pupper puppo)

solution:
- create column type that holds this value 
- drop those columns 

In [149]:
id_list = list(twitter_acrchive_clean.columns)
dog_types = ['doggo','floofer','pupper','puppo']
id_list = [x for x in id_list if x not in dog_types]

dog_type = twitter_acrchive_clean.melt(id_vars=id_list,value_name='dog_type')
dog_type = dog_type[['tweet_id','dog_type']]

In [150]:

dog_type = dog_type[dog_type.dog_type != 'None']
twitter_acrchive_clean = twitter_acrchive_clean.merge(dog_type,on='tweet_id',how='left')

In [151]:
twitter_acrchive_clean.drop(columns=dog_types, inplace=True)

# test

In [152]:
twitter_acrchive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2368 entries, 0 to 2367
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2368 non-null   int64  
 1   in_reply_to_status_id       79 non-null     float64
 2   in_reply_to_user_id         79 non-null     float64
 3   timestamp                   2368 non-null   object 
 4   source                      2368 non-null   object 
 5   text                        2368 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2309 non-null   object 
 10  rating_numerator            2368 non-null   int64  
 11  rating_denominator          2368 non-null   int64  
 12  name                        2368 non-null   object 
 13  retweet_count               2368 

## dirty data
* the count increased from `2356` to `2368` 
    - some dogs has multipe types

## column source is not descriptive
- rename to `source_device`

In [156]:
twitter_acrchive_clean.rename(columns={'source':'source_device'}, inplace=True)

In [157]:

twitter_acrchive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2368 entries, 0 to 2367
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2368 non-null   int64  
 1   in_reply_to_status_id       79 non-null     float64
 2   in_reply_to_user_id         79 non-null     float64
 3   timestamp                   2368 non-null   object 
 4   source_device               2368 non-null   object 
 5   text                        2368 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2309 non-null   object 
 10  rating_numerator            2368 non-null   int64  
 11  rating_denominator          2368 non-null   int64  
 12  name                        2368 non-null   object 
 13  retweet_count               2368 